<a href="https://colab.research.google.com/github/LucasCloud777/AWS-CLI/blob/main/docs/ais-templates/aistudio_gemini_prompt_freeform_nofiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [3]:
!pip install -U -q "google-generativeai>=0.8.2"

In [10]:
!apt-get install nmap -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblinear4 liblua5.3-0 libpcap0.8 lua-lpeg nmap-common
Suggested packages:
  liblinear-tools liblinear-dev ncat ndiff zenmap
The following NEW packages will be installed:
  liblinear4 liblua5.3-0 libpcap0.8 lua-lpeg nmap nmap-common
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 6,029 kB of archives.
After this operation, 26.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 liblinear4 amd64 2.3.0+dfsg-5 [41.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblua5.3-0 amd64 5.3.6-1build1 [140 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lua-lpeg amd64 1.0.2-1 [31.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy

In [4]:
# import necessary modules.

import google.generativeai as genai

import base64
import json

try:
    # Mount google drive
    from google.colab import drive

    drive.mount("/gdrive")

    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = b'W3sicGFydHMiOiBbeyJ0ZXh0IjogIkhlbGxvIn1dfV0='
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

print(json.dumps(contents, indent=4))

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
[
    {
        "parts": [
            {
                "text": "Hello"
            }
        ]
    }
]


## Call `generate_content`

In [24]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [51]:
import google.generativeai as genai
import base64
import json
import os
from google.colab import drive, userdata
import re
import subprocess  # Para executar comandos do Nmap
import ipywidgets as widgets
from IPython.display import display, Markdown
import threading
import time

# Montar o Google Drive
drive.mount("/gdrive", force_remount=True)

# Obter a chave da API do Colab Secrets
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Verificar se a chave foi carregada
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("A chave da API não foi encontrada. Verifique se o segredo foi configurado corretamente.")

# Configurar o SDK do genai
genai.configure(api_key=api_key)

# Função para validar IP ou domínio
def validar_input(alvo):
    ip_pattern = re.compile(r"^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$")  # Para validar IPv4
    dominio_pattern = re.compile(
        r"^(?=.{1,255}$)(?!-)([A-Za-z0-9-]{1,63}\.)+[A-Za-z]{2,63}$"
    )  # Para validar domínios
    if ip_pattern.match(alvo) or dominio_pattern.match(alvo):
        return True
    else:
        return False

# Lista avançada de scripts disponíveis
scripts_disponiveis = [
    "http-enum", "ssl-cert", "dns-brute", "ftp-anon",
    "vulners", "smb-os-discovery", "ssh-auth-methods",
    "http-title", "dns-brute.nse", "hostmap-bfk.nse",
    "traceroute-geolocation.nse", "http-enum.nse",
    "default", "safe", "auth", "broadcast",
    "brute", "discovery", "dos", "exploit",
    "external", "fuzzer", "intrusive", "malware",
    "version", "vuln", "whois-domain", "ftp-syst",
    "http-methods", "smtp-open-relay", "snmp-interfaces",
    "mysql-info", "smb-vuln-ms17-010", "rdp-enum-encryption"
]

# Widgets para a interface gráfica
scripts_label = widgets.Label("Escolha os scripts que deseja usar (Ctrl+Clique para selecionar múltiplos):")
scripts_selecao = widgets.SelectMultiple(options=scripts_disponiveis, description="Scripts:")
alvo_input = widgets.Text(description="IP/Domínio:", placeholder="Digite o IP ou domínio")
botao_analisar = widgets.Button(description="Analisar")
resultado_output = widgets.Output()

# Variável global para controle de animação de carregamento
carregando = False

# Função para exibir a animação de carregamento
def exibindo_carregamento():
    simbolos = ["|", "/", "-", "\\"]
    idx = 0
    while carregando:
        print(f"\rIniciando análise avançada com HORUS AI... {simbolos[idx]}", end="")
        idx = (idx + 1) % len(simbolos)
        time.sleep(0.2)

# Função para processar os inputs
def executar_analise(b):
    global carregando
    with resultado_output:
        resultado_output.clear_output()

        # Obter os scripts selecionados
        scripts_selecionados = list(scripts_selecao.value)
        if not scripts_selecionados:
            print("Por favor, selecione pelo menos um script.")
            return

        # Obter o alvo
        alvo = alvo_input.value.strip()
        if not validar_input(alvo):
            print("Input inválido! Por favor, insira um IP ou domínio válido.")
            return

        # Montar o comando do Nmap
        comando_nmap = ["nmap", "-sV", alvo]
        for script in scripts_selecionados:
            comando_nmap.extend(["--script", script])

        # Iniciar animação de carregamento
        carregando = True
        animacao_thread = threading.Thread(target=exibindo_carregamento)
        animacao_thread.start()

        # Executar o Nmap
        try:
            nmap_result = subprocess.check_output(comando_nmap, text=True)

            # Parar animação
            carregando = False
            animacao_thread.join()

            print("\n--- Resultado do Nmap ---\n")
            print(nmap_result)
# Preparar o conteúdo para análise
conteudo_para_analise = [{"parts": [{"text": nmap_result}]}]

# Instruções detalhadas para análise de segurança cibernética
generation_config = {
    "instructions": """Analise os resultados do Nmap e forneça uma análise detalhada de segurança cibernética.
                       - Identifique vulnerabilidades críticas nas portas abertas.
                       - Verifique se há protocolos inseguros em execução (ex: HTTP em vez de HTTPS).
                       - Sugira práticas de mitigação e recomendações para fortalecer a segurança.
                       - Se algum serviço vulnerável for encontrado, forneça informações sobre como realizar uma correção ou mitigação adequada."""
}

# Chamar o modelo e exibir a resposta
gemini = genai.GenerativeModel(model_name=model)
response = gemini.generate_content(
    conteudo_para_analise,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))


        except FileNotFoundError:
            carregando = False
            animacao_thread.join()
            print("O Nmap não está instalado ou não está configurado no ambiente.")
        except subprocess.CalledProcessError as e:
            carregando = False
            animacao_thread.join()
            print("Erro ao executar o Nmap:", e)

# Associar a função ao botão
botao_analisar.on_click(executar_analise)

# Exibir os widgets
display(scripts_label, scripts_selecao, alvo_input, botao_analisar, resultado_output)


Mounted at /gdrive


Label(value='Escolha os scripts que deseja usar (Ctrl+Clique para selecionar múltiplos):')

SelectMultiple(description='Scripts:', options=('http-enum', 'ssl-cert', 'dns-brute', 'ftp-anon', 'vulners', '…

Text(value='', description='IP/Domínio:', placeholder='Digite o IP ou domínio')

Button(description='Analisar', style=ButtonStyle())

Output()

In [52]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

2025-01-22 08:20:19.438 200 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3792.58ms


This Nmap output indicates that the target IP address appears to be offline or unresponsive.  The scan completed quickly (3.45 seconds) and reported zero hosts up.  The "Host seems down" message suggests that Nmap's standard ping (ICMP echo request) probes failed to elicit a response.

The suggestion to use the `-Pn` flag is important.  This tells Nmap to *skip* the ping scan and assume the host is up.  This is useful because some firewalls or security systems actively block ICMP ping requests, even if the host is otherwise accessible on other ports.  If you suspect this is the case, running `nmap -Pn <target_ip_address>` will attempt to scan the ports directly without relying on ICMP.  If the host is actually up, this should produce more meaningful results (though it will take longer).


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>